In [3]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import numpy as np
from qiskit import ClassicalRegister

# --- Config ---
BOARD_SIZE = 4  # total number of squares (must be a perfect square for display)
shots = 4096
simulator = Aer.get_backend('qasm_simulator')

# --- Game State ---
measured_squares = {}  # square_idx -> 0 (safe) or 1 (mine)
game_over = False

# Create quantum board in equal superposition
qc = QuantumCircuit(BOARD_SIZE, BOARD_SIZE)
for i in range(BOARD_SIZE):
    qc.h(i)  # superposition of safe (0) and mine (1)


# --- Helper functions ---
def board_to_string():
    """Return a string showing the current classical board."""
    out = ""
    side_len = int(BOARD_SIZE ** 0.5)
    for i in range(BOARD_SIZE):
        if i in measured_squares:
            out += "* " if measured_squares[i] == 1 else "0 "
        else:
            out += ". "
        if (i + 1) % side_len == 0:
            out += "\n"
    return out


def probability_board(qc_state):
    """Return 2D list with per-square mine probabilities."""
    qc_copy = qc_state.copy()
    qc_copy.measure_all()
    result = simulator.run(qc_copy, shots=shots).result()
    counts = result.get_counts()

    prob_per_square = np.zeros(BOARD_SIZE)
    for bitstring, freq in counts.items():
        bits = bitstring[::-1]  # reverse to match qubit numbering
        for idx in range(BOARD_SIZE):
            if bits[idx] == '1':
                prob_per_square[idx] += freq

    prob_per_square /= shots
    side_len = int(BOARD_SIZE ** 0.5)
    prob_grid = prob_per_square.reshape((side_len, side_len))
    return prob_grid, counts


def print_probability_board(prob_grid):
    """Nicely print the probability board."""
    for row in prob_grid:
        print(" ".join(f"{p:.2f}" for p in row))


def plot_board_state(counts):
    """Plot histogram of full board state probabilities."""
    plot_histogram(counts)
    plt.show()

def check_square(qc_state, square_idx):
    """Measure a square, collapse state, update board, and return new state circuit."""
    global game_over

    if square_idx in measured_squares:
        print(f"Square {square_idx} already revealed!")
        return qc_state

    print(f"Checking square {square_idx}...")

    qc_measure = qc_state.copy()
    qc_measure.add_register(ClassicalRegister(1))  # Add 1 classical bit
    qc_measure.measure(square_idx, 0)
    result = simulator.run(qc_measure, shots=1).result()
    outcome = list(result.get_counts().keys())[0]
    state = int(outcome[::-1][0])  # The single classical bit is our result

    measured_squares[square_idx] = state

    if state == 1:
        print(f"💣 BOOM! Mine at square {square_idx}. Game over.")
        game_over = True
    else:
        print(f"✅ Safe at square {square_idx}.")

    # Collapse: create a new state circuit conditioned on the outcome
    new_qc = QuantumCircuit(BOARD_SIZE)
    for i in range(BOARD_SIZE):
        if i in measured_squares:
            if measured_squares[i] == 1:
                new_qc.x(i)  # set to |1>
            # else default is |0>
        else:
            new_qc.h(i)  # still superposition
    return new_qc


# --- Game Loop ---
print("Welcome to Quantum Minesweeper!")
print("Squares are numbered 0 to", BOARD_SIZE - 1)

while not game_over and len(measured_squares) < BOARD_SIZE:
    print("\nClassical Board:")
    print(board_to_string())

    prob_grid, counts = probability_board(qc)
    print("Mine Probability Board:")
    print_probability_board(prob_grid)

    plot_board_state(counts)

    try:
        move = int(input(f"Enter square to check (0-{BOARD_SIZE-1}): "))
        if move < 0 or move >= BOARD_SIZE:
            print("Invalid square number.")
            continue
        qc = check_square(qc, move)
    except ValueError:
        print("Please enter a valid number.")

if not game_over:
    print("🎉 Congratulations, you cleared the board!")

print("\nFinal Board:")
print(board_to_string())


Welcome to Quantum Minesweeper!
Squares are numbered 0 to 3

Classical Board:
. . 
. . 

Mine Probability Board:
0.00 0.00
0.00 0.00
Please enter a valid number.

Classical Board:
. . 
. . 

Mine Probability Board:
0.00 0.00
0.00 0.00
Please enter a valid number.

Classical Board:
. . 
. . 

Mine Probability Board:
0.00 0.00
0.00 0.00
Please enter a valid number.

Classical Board:
. . 
. . 

Mine Probability Board:
0.00 0.00
0.00 0.00
Please enter a valid number.

Classical Board:
. . 
. . 

Mine Probability Board:
0.00 0.00
0.00 0.00
Checking square 1...
✅ Safe at square 1.

Classical Board:
. 0 
. . 

Mine Probability Board:
0.51 0.00
0.49 0.49
Checking square 2...
✅ Safe at square 2.

Classical Board:
. 0 
0 . 

Mine Probability Board:
0.50 0.00
0.00 0.51
Checking square 0...
💣 BOOM! Mine at square 0. Game over.

Final Board:
* 0 
0 . 

